# Day 28 – Advanced Visualization
This notebook demonstrates how to build the reusable charts from `advanced_visualization.py`.

In [ ]:
import pandas as pd
from Day_28_Advanced_Visualization import advanced_visualization as av

## Create a demo dataset
In a real workflow you would call `av.load_sales_data()`.
For the walkthrough we will use a compact dataset.

In [ ]:
sales_df = pd.DataFrame(
    {
        'Date': pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-02', '2024-01-03']),
        'Product': ['A', 'A', 'B', 'B'],
        'Revenue': [100.0, 120.0, 80.0, 140.0],
    }
)
sales_df

## Build the product revenue bar chart

In [ ]:
fig_bar = av.build_product_revenue_bar(sales_df)
fig_bar

## Build the sales dashboard

In [ ]:
fig_dashboard = av.build_sales_dashboard(sales_df)
fig_dashboard